In [ ]:
from sklearn.datasets import load_iris
import numpy as np

iris = load_iris()
X = iris.data   # features
y = iris.target # actual species labels (0=setosa, 1=versicolor, 2=virginica)


In [2]:
from sklearn.preprocessing import MinMaxScaler
import skfuzzy as fuzz

# Normalize features to [0,1]
scaler = MinMaxScaler()
X_norm = scaler.fit_transform(X)

# Transpose for skfuzzy (shape: features x samples)
data = X_norm.T

# Apply FCM
cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(
    data, c=3, m=2, error=0.005, maxiter=1000, init=None)

# Predicted cluster = highest membership
predicted_clusters = np.argmax(u, axis=0)


In [3]:
print("Predicted clusters for first 20 samples:", predicted_clusters[:20])


Predicted clusters for first 20 samples: [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


In [4]:
print("Actual labels for first 20 samples:", y[:20])


Actual labels for first 20 samples: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [5]:
from scipy.stats import mode

mapping = {}
for cluster_id in range(3):
    mask = predicted_clusters == cluster_id
    if np.any(mask):
        mapping[cluster_id] = mode(y[mask], keepdims=True).mode[0]

mapped_preds = np.array([mapping[c] for c in predicted_clusters])

accuracy = np.mean(mapped_preds == y)
print("FCM Accuracy:", accuracy)


FCM Accuracy: 0.8933333333333333


In [6]:
print("Fuzzy Partition Coefficient (FPC):", fpc)


Fuzzy Partition Coefficient (FPC): 0.7424800003779325


In [7]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=42).fit(X_norm)
kmeans_preds = kmeans.labels_

# Majority mapping for K-means
mapping_kmeans = {}
for cluster_id in range(3):
    mask = kmeans_preds == cluster_id
    if np.any(mask):
        mapping_kmeans[cluster_id] = mode(y[mask], keepdims=True).mode[0]

mapped_kmeans = np.array([mapping_kmeans[c] for c in kmeans_preds])
accuracy_kmeans = np.mean(mapped_kmeans == y)

print("K-means Accuracy:", accuracy_kmeans)


K-means Accuracy: 0.88
